In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [4]:
pg_url = 'postgresql://root:root@pg_database:5432/ny_taxi'

def load_file(file_path: str, table_name: str, pg_url: str) -> None:
    if file_path.endswith('.parquet'):
        df = pd.read_parquet(file_path)
    elif file_path.endswith('.csv') or file_path.endswith('.csv.gz'):
        df = pd.read_csv(file_path)
    else:
        return
    df.columns = [c.lower() for c in df.columns] # postgres doesn't like capitals or spaces
    engine = create_engine(pg_url)
    df.to_sql(name=table_name, con=engine, if_exists='replace', chunksize=100000)

In [5]:
create_engine(pg_url).connect()

In [6]:
!ls data

data_dictionary_trip_records_yellow.pdf  yellow_tripdata_2021-01.csv.gz
green_tripdata_2019_01.csv.gz		 yellow_tripdata_2021_01.parquet
taxi_zone_lookup.csv


In [23]:
load_file('data/yellow_tripdata_2021_01.parquet', 'f_trip', pg_url)

   vendorid tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0         1  2021-01-01 00:30:10   2021-01-01 00:36:12              1.0   
1         1  2021-01-01 00:51:20   2021-01-01 00:52:19              1.0   
2         1  2021-01-01 00:43:30   2021-01-01 01:11:06              1.0   
3         1  2021-01-01 00:15:48   2021-01-01 00:31:01              0.0   
4         2  2021-01-01 00:31:49   2021-01-01 00:48:21              1.0   

   trip_distance  ratecodeid store_and_fwd_flag  pulocationid  dolocationid  \
0           2.10         1.0                  N           142            43   
1           0.20         1.0                  N           238           151   
2          14.70         1.0                  N           132           165   
3          10.60         1.0                  N           138           132   
4           4.94         1.0                  N            68            33   

   payment_type  fare_amount  extra  mta_tax  tip_amount  tolls_amount  \


In [24]:
load_file('data/taxi_zone_lookup.csv', 'd_zone', pg_url)

   locationid        borough                     zone service_zone
0           1            EWR           Newark Airport          EWR
1           2         Queens              Jamaica Bay    Boro Zone
2           3          Bronx  Allerton/Pelham Gardens    Boro Zone
3           4      Manhattan            Alphabet City  Yellow Zone
4           5  Staten Island            Arden Heights    Boro Zone


In [7]:
load_file('data/green_tripdata_2019_01.csv.gz', 'green_tripdata_2019_01', pg_url)